In [1]:
import pandas as pd
import numpy as np
import datetime
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last

# CH11时间序列

In [14]:
# time模块
# calendar模块


# datetime模块：比time模块更高级
## datetime模块中的类
### datetime
import time
#### 获取时间
##### 获取时间-获取当前时间
now = datetime.datetime.now()#datetime.datetime以毫秒形式存储日期和时间
print(now)
#####获取时间-获取指定时间
birth = datetime.datetime(1996,4,27,1,30)
birth

2019-05-26 11:42:37.856881


datetime.datetime(1996, 4, 27, 1, 30)

In [30]:
#### 时间相减 获取时间间隔对象
time.sleep(3)
now2 = datetime.datetime.now()
delta = now2-now
delta
delta.days
delta.seconds

delta2 = datetime.datetime(2019,5,26,11,35)-datetime.datetime(1996,4,27,1,30)
delta2.total_seconds()

datetime.timedelta(seconds=10871, microseconds=400093)

0

10871

728301900.0

In [18]:
### timedelta
#### 可以给datetime对象加或减一个或多个timedelta
start = datetime.datetime(2011,1,7)
start+datetime.timedelta(10)#加上10天
start-2*datetime.timedelta(10)

datetime.datetime(2011, 1, 17, 0, 0)

datetime.datetime(2010, 12, 18, 0, 0)

In [19]:
###tzinfo类
datetime.tzinfo()

In [36]:
###date类
a = datetime.date(2019,5,26)-datetime.date(2000,1,3)
a
a.days
testday = datetime.date(2017,9,4)
testday.isocalendar()
testday.timetuple()

datetime.timedelta(days=7083)

7083

(2017, 36, 1)

time.struct_time(tm_year=2017, tm_mon=9, tm_mday=4, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=247, tm_isdst=-1)

### 字符串和datetime的互相转换

In [38]:
#str
stamp = datetime.datetime(2011,1,3)
str(stamp)
#strftime(传入格式化字符串)
stamp.strftime('%Y-%m-%d')

'2011-01-03 00:00:00'

'2011-01-03'

In [43]:
#datetime.strptime()
value = '2011-01-03'
datetime.datetime.strptime(value,'%Y-%m-%d')

datestrs = ['7/6/2011','8/6/2011']
[datetime.datetime.strptime(x,'%m/%d/%Y') for x in datestrs]

datetime.datetime(2011, 1, 3, 0, 0)

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [50]:
#dateutil的parser.parse
from dateutil.parser import parse
parse('2011-01-03')
parse('Jan 31,1997 10:45 PM')
parse('6/12/2011',dayfirst=True)#日在月前

datetime.datetime(2011, 1, 3, 0, 0)

datetime.datetime(2019, 1, 31, 22, 45)

datetime.datetime(2011, 12, 6, 0, 0)

In [56]:
#pandas的to_datetime
datestrs = ['2011-07-06 12:00:00','2011-08-06 00:00:00']
pd.to_datetime(datestrs)
a=pd.to_datetime(datestrs+[None])
a[2]
pd.isnull(a)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

NaT

array([False, False,  True])

## 11.2时间序列基础

### pandas最基础的时间序列类型：以时间戳为索引的Series


In [ ]:
from datetime import datetime
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),datetime(2011, 1, 7), 
         datetime(2011, 1, 8),datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6),index=dates)
ts
ts.index
ts.index[0]
ts[::2]
ts+ts[::2]

### 索引、选取、子集构造

In [77]:
#根据索引选取数据
ts
stamp = ts.index[2] 
ts[stamp]
#简化：传入一个日期字符串
ts['1/10/2011']
ts['20110110']
#较长时间序列：传入‘年’或‘年月’选取数据切片
long_ts = pd.Series(np.random.randn(1000),index=pd.date_range('1/1/2000',periods=1000))
# long_ts['2001']
# long_ts['2001-01']
#datetime对象切片
ts[datetime(2011,1,7):]
#datetime对象切片:不存在于该时间序列中的时间戳对其进行切片
ts['1/6/2011':'1/11/2011']
ts
#等价的实例方法截取两个日期之间的TimeSeries
ts.truncate(before='2011-01-06',after='2011-01-11')

2011-01-02    0.186876
2011-01-05    1.538538
2011-01-07   -0.372678
2011-01-08    0.922825
2011-01-10    0.987718
2011-01-12   -0.415472
dtype: float64

-0.3726777149049621

0.9877176180684903

0.9877176180684903

2011-01-07   -0.372678
2011-01-08    0.922825
2011-01-10    0.987718
2011-01-12   -0.415472
dtype: float64

2011-01-07   -0.372678
2011-01-08    0.922825
2011-01-10    0.987718
dtype: float64

2011-01-02    0.186876
2011-01-05    1.538538
2011-01-07   -0.372678
2011-01-08    0.922825
2011-01-10    0.987718
2011-01-12   -0.415472
dtype: float64

2011-01-07   -0.372678
2011-01-08    0.922825
2011-01-10    0.987718
dtype: float64

In [ ]:
# dateframe
dates = pd.date_range('1/1/2000',periods=100,freq='W-WED')#频率是每周的每周三
long_df = pd.DataFrame(np.random.randn(100,4),index=dates,columns=['Colorado','Texas','New York','Ohio'])
long_df.loc['2001-05']

### 带有重复索引的时间序列

In [83]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000'])
dup_ts=pd.Series(np.arange(5),index=dates)
dup_ts
#索引
##时间点不重复：产生标量值
dup_ts['1/3/2000']
##时间点重复：产生切片
dup_ts['1/2/2000']
#对非唯一时间戳聚合：groupby，再level=0
grouped = dup_ts.groupby(level=0)


2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

4

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

## 11.3日期的范围、频率以及移动 

### 生成日期范围 

In [87]:
#pandas.date_range 可用于 根据指定的频率生成指定长度的 DatetimeIndex：
index = pd.date_range('2012-04-01','2012-06-01') #默认按天产生
index
pd.date_range(start='2012-04-01', periods=20)#只传入起始或者结束还要再传入一个period
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [88]:
pd.date_range('2000-01-01',periods=3,freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31'], dtype='datetime64[ns]', freq='BM')

In [93]:
pd.date_range('2012-05-02 12:56:31', periods=5)#保留时间戳的时间信息
pd.date_range('2012-05-02 12:56:31', periods=5,normalize = True)#规范化时间戳

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移量

In [96]:
#给freq传入字符串
pd.date_range('2000-01-01','2000-01-03 23:59',freq='4h')
#偏移量对象可由加法连接
from pandas.tseries.offsets import Hour,Minute
Hour(2)+Minute(30)
#传入频率字符串
pd.date_range('2000-01-01',periods=10,freq='1h30min')
#非均匀分割的时间点：月末'M'月工作日末'BM'
pd.date_range('2000-01-01',periods=10,freq='2M')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

<150 * Minutes>

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

DatetimeIndex(['2000-01-31', '2000-03-31', '2000-05-31', '2000-07-31',
               '2000-09-30', '2000-11-30', '2001-01-31', '2001-03-31',
               '2001-05-31', '2001-07-31'],
              dtype='datetime64[ns]', freq='2M')

### WOM日期（week of month）

In [97]:
pd.date_range('2012-01-01','2012-09-01',freq='WOM-3FRI')#第三个礼拜五

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

### 移动（超前和滞后）数据

In [103]:
# 
ts = pd.Series(np.random.randn(4),index=pd.date_range('1/1/2000', periods=4,freq='M'))
ts
ts.shift(2)#数据向下移2
ts.shift(-2)#数据向上移2
#计算时间序列的百分比变化
ts/ts.shift(1) - 1
#时间戳位移
ts
ts.shift(2,freq='M')#时间都加了两个月
ts.shift(3,freq='D')#时间都加了3天

2000-01-31    1.111574
2000-02-29    3.522140
2000-03-31    0.010357
2000-04-30    0.154970
Freq: M, dtype: float64

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    1.111574
2000-04-30    3.522140
Freq: M, dtype: float64

2000-01-31    0.010357
2000-02-29    0.154970
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

2000-01-31          NaN
2000-02-29     2.168607
2000-03-31    -0.997059
2000-04-30    13.962183
Freq: M, dtype: float64

2000-01-31    1.111574
2000-02-29    3.522140
2000-03-31    0.010357
2000-04-30    0.154970
Freq: M, dtype: float64

2000-03-31    1.111574
2000-04-30    3.522140
2000-05-31    0.010357
2000-06-30    0.154970
Freq: M, dtype: float64

### 通过偏移量对日期进行位移

In [116]:
from pandas.tseries.offsets import Day,MonthEnd
now = datetime(2017,11,17)
#偏移量作用在datetime和MonthEnd对象上
now+3*Day()
#锚点偏移量：第一次增量将原日期向前滚动到复合频率规则的下一个日期
now+MonthEnd()
now+MonthEnd(1)
now+MonthEnd(3)


Timestamp('2017-11-20 00:00:00')

Timestamp('2017-11-30 00:00:00')

Timestamp('2017-11-30 00:00:00')

Timestamp('2018-01-31 00:00:00')

Timestamp('2017-11-30 00:00:00')

In [124]:
#锚点偏移量的rollback和rollforward，将日期向后向前滚
offset = MonthEnd()#锚点偏移量选择MonthEnd
offset.rollforward(now)#锚点偏移量的rollforward 作用在datetime上
offset.rollback(now)
#结合锚点偏移量的滚动方法和groupby
ts = pd.Series(np.random.randn(20),index=pd.date_range('1/15/2000',periods=20,freq='4d'))
ts
ts.groupby(offset.rollforward).mean()

Timestamp('2017-11-30 00:00:00')

Timestamp('2017-10-31 00:00:00')

2000-01-15   -0.832539
2000-01-19    1.520574
2000-01-23   -1.106051
2000-01-27   -0.773002
2000-01-31    1.636820
2000-02-04    1.246884
2000-02-08    0.590459
2000-02-12    0.185175
2000-02-16    0.440332
2000-02-20   -0.575971
2000-02-24    0.885716
2000-02-28    0.506866
2000-03-03    1.217959
2000-03-07   -0.291374
2000-03-11    0.089142
2000-03-15    0.513137
2000-03-19    0.934920
2000-03-23    0.518773
2000-03-27   -1.697291
2000-03-31   -0.597165
Freq: 4D, dtype: float64

2000-01-31    0.089160
2000-02-29    0.468494
2000-03-31    0.086013
dtype: float64

## 11.5时期及其算数运算

In [130]:
p = pd.Period(2007,freq='A-DEC')
p
#Period对象加减整数：根据频率位移
p+6
#相同频率的Period对象相减：二者之间的单位数量
pd.Period('2016',freq='A-DEC')- p

Period('2007', 'A-DEC')

Period('2013', 'A-DEC')

9

In [135]:
#创建时期范围
rng = pd.period_range('2000-01-01','2000-06-03',freq='M')#返回PeriodIndex对象
#PeriodIndex保存了一组Period，可以在任何pd数据结构中被用作轴索引
pd.Series(np.random.randn(6),index=rng)
#一个字符串数组转为eriodIndex
values = ['2001Q3','2002Q2', '2003Q1']
index = pd.PeriodIndex(values,freq='Q-DEC')
index

2000-01    0.326127
2000-02    0.334477
2000-03    0.141829
2000-04    0.314344
2000-05   -0.137527
2000-06   -0.529511
Freq: M, dtype: float64

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### 时期的频率转换

In [140]:
#asfreq转变
#Period对象转变
#年度转月度
##以12月为结束
p = pd.Period('2007',freq='A-DEC')
p.asfreq('M',how='start')
p.asfreq('M',how='end')
##不以12月为结束
p = pd.Period('2007',freq='A-JUN')
p.asfreq('M',how='start')#2006年7月起
p.asfreq('M',how='end')#2007年6月止

Period('2007-01', 'M')

Period('2007-12', 'M')

Period('2006-07', 'M')

Period('2007-06', 'M')

In [141]:
#月度转年度(高频率转低频率)
p = pd.Period('2007-08',freq='M')
p.asfreq('A-DEC')
p.asfreq('A-JUN')#由2007-08所属位置决定

Period('2007', 'A-DEC')

Period('2008', 'A-JUN')

In [145]:
#PeriodIndex对象转变
rng = pd.period_range('2006','2009',freq='A-DEC')
ts = pd.Series(np.random.randn(len(rng)),index=rng)
ts
ts.asfreq('M',how='start')#年度时期的第一个月
ts.asfreq('B',how='end')#每年的最后一个工作日

2006   -0.599404
2007   -1.370762
2008    0.218697
2009   -0.862285
Freq: A-DEC, dtype: float64

2006-01   -0.599404
2007-01   -1.370762
2008-01    0.218697
2009-01   -0.862285
Freq: M, dtype: float64

2006-12-29   -0.599404
2007-12-31   -1.370762
2008-12-31    0.218697
2009-12-31   -0.862285
Freq: B, dtype: float64